### BERT model for music emotion classification.

In [1]:
import pandas as pd
import numpy as np

import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [2]:
# load the data

with open("../data/train/lyrics/lyrics.txt", "r") as f:
    lyrics = f.read()

with open("../data/train/lyrics/labels.txt", "r") as f:
    labels = f.read()

In [3]:
lyrics_split = lyrics.split("\n")
labels_split = labels.split("\n")
lyrics_split.remove('')
labels_split.remove('')

print(lyrics_split[0], "\n",labels_split[0])

Gently hold our hands Gently hold our heads on high  Aimless time in fear new hide Overthrow the plan Confusion lies in all my words Mad is the soul  We barricade ourselves in holes of temperament This is the dawning of a new age A heart that beats the wrong way Insanity's crescendo  Windcolour, second sight A touch of silence and the violence of dark Illusion span, the aroma of time Shadowlife and the scent of nothingness  Infinite fall of instinct Order of one spells deceit Infin 
 1


In [4]:
data = pd.DataFrame({"Lyrics": lyrics_split,
                     "Quadrant": labels_split})

data.head()

,Lyrics,Quadrant
0,Gently hold our hands Gently hold our heads on...,1
1,We are the Sun We are the dead stars We are th...,1
2,You're out of touch I'm out of time But I'm ou...,0
3,You finally close the door You've left open wi...,0
4,"Lullaby by birdland that's what I Always hear,...",0


In [5]:
data["Quadrant"] = pd.to_numeric(data["Quadrant"])

In [6]:
# labels or quadrants already encoded
data["Quadrant"].value_counts()

0    137
2     76
1     64
3     60
Name: Quadrant, dtype: int64

### Train / Validation Split

In [7]:
X_train, X_val, y_train, y_val = train_test_split(data.index.values,
                                                  data.Quadrant.values,
                                                  test_size=0.15,
                                                  random_state=42,
                                                  stratify=data.Quadrant.values)

data["data_type"] = ["not_set"]*data.shape[0]

data.loc[X_train, "data_type"] = "train"
data.loc[X_val, "data_type"] = "val"
data.groupby(["Quadrant", "data_type"]).count()

Lyrics
Quadrant data_type        
0        train         116
         val            21
1        train          54
         val            10
2        train          65
         val            11
3        train          51
         val             9

### Tokenize and Encode the Data

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)

encoded_data_train = tokenizer.batch_encode_plus(
    data[data.data_type=="train"].Lyrics.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors="pt"
)

encoded_data_val = tokenizer.batch_encode_plus(
    data[data.data_type=="val"].Lyrics.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors="pt"
)

ValueError: Connection error, and we cannot find the requested files in the cached path. Please try again or make sure your Internet connection is on.

In [ ]:
input_id_train = encoded_data_train["input_ids"]
attention_masks_train = encoded_data_train["attention_mask"]
labels_train = torch.tensor(data[data.data_type == "train"].Quadrant.values)


input_id_val = encoded_data_val["input_ids"]
attention_masks_val = encoded_data_val["attention_mask"]
labels_val = torch.tensor(data[data.data_type == "val"].Quadrant.values)

# datasets
trainset = TensorDataset(input_id_train, attention_masks_train, labels_train)
validationset = TensorDataset(input_id_val, attention_masks_val, labels_val)

### BERT Pre-trained Model

In [42]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=4,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [45]:
# dataloaders
trainloader = DataLoader(trainset, sampler=RandomSampler(trainset), batch_size=3)
validationloader = DataLoader(validationset, sampler=SequentialSampler(validationset), batch_size=3)

In [46]:
# optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)
epochs = 5
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(trainloader)*epochs)

In [47]:
# performance metrics
def f1_score_func(preds, labels):
    pred_flat = np.argmax(preds, 1).flatten()
    labels_flat = labels.flatten()

    return f1_score(labels_flat, pred_flat, average="weighted")

def accuracy_per_class(preds, labels):
    quadrant_dict = {0: "Q1", 1: "Q2", 2: "Q3", 3: "Q4"}

    pred_flat = np.argmax(preds, 1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = pred_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        print(f"Quadrant: {quadrant_dict[label]}")
        print(f"Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n")

### Training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed(seed_val)

def evaluate(validation_loader):
    model.eval()

    total_loss = 0
    y_pred, y_true = [], []

    for batch in validation_loader:
        batch = tuple(b.to(device) for b in batch)

        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "labels": batch[2]
                  }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        total_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs["labels"].cpu().numpy()
        y_pred.append(logits)
        y_true.append(label_ids)

    loss_avg = total_loss / len(validation_loader)
    predictions = np.concatenate(y_pred, axis=0)
    true_vals = np.concatenate(y_true, axis=0)

    return loss_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()

    total_loss = 0

    progress_bar = tqdm(trainloader,
                        desc="Epoch {:1d}".format(epochs),
                        leave=False,
                        disable=False)

    for batch in progress_bar:
        batch = tuple(b.to(device) for b in batch)

        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "labels": batch[2]
                  }

        model.zero_grad()
        output = model(**inputs)
        loss = output[0]
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({"training_loss: {:.3f}".format(loss.item()/len(batch))})

    torch.save(model.state_dict(), f"finetuned_BERT_epoch_{epoch}.pt")

    tqdm.write(f"\nEpoch {epoch}")

    loss_avg = total_loss / len((trainloader))
    tqdm.write(f"Training Loss: {loss_avg}")

    val_loss, predictions, true_vals = evaluate(validationloader)
    val_f1_score = f1_score_func(predictions, true_vals)
    tqdm.write(f"Validation Loss: {val_loss}")
    tqdm.write(f"F1 Score (Weighted): {val_f1_score}')")

### Loading and Evalutaion

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=4,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
model.to(device)
model.load_state_dict(torch.load("finetuned_BERT_epoch_1.pt", map_location="cpu"))
_, predictions, true_vals = evaluate(validationloader)
accuracy_per_class(predictions, true_vals)

### Resources

https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613
